In [13]:
import urllib.request as ur
import re 

f = ur.urlopen('https://resultados.as.com/resultados/futbol/primera/jornada')
s = f.read().decode()


In [40]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [16]:
from pprint import pprint     # import para hacer prints boninitos 

#Obtener url de cada jornada 
urlJornadas = re.findall('<li>.*?cont-jornada" href="(.*?)".*?</li>',s,re.DOTALL)

pprint(urlJornadas)

['/resultados/futbol/primera/2018_2019/jornada/regular_a_1/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_2/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_3/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_4/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_5/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_6/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_7/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_8/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_9/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_10/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_11/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_12/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_13/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_14/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_15/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_16/',
 

In [106]:
import datetime

now = datetime.datetime.now()
months = ['Ago','Sep','Oct','Nov','Dic','Ene','Feb','Mar','Abr','May','Jun','Jul'] #Inicia en agosto como la temporada 
day = now.day
month = (now.month + 4) % 12

#Abrir la jornada correspondiente para cada url 
'''for x in urlJornadas:
    f = ur.urlopen("https://resultados.as.com"+x)
    s = f.read().decode()
    print(s + "\n")'''

f = ur.urlopen("https://resultados.as.com/resultados/futbol/primera/2018_2019/jornada/regular_a_26/")
jornadai = f.read().decode()

#Obtener número de jornada y fecha 
jornadaFecha = re.findall('<h2 class="tit-modulo">.*?(Jornada \d*).*?>(\d\d) *-* *(\d\d)* (.*?)<.*?</h2>',jornadai,re.DOTALL)
print(jornadaFecha)


def printFuture(jornadaFecha): 
    if jornadaFecha[0][2] != '':
        print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][2] +" de " + jornadaFecha[0][3] )
    else: 
        print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][3] )

def printPast(jornadaFecha):
    if jornadaFecha[0][2] != '':
        print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][2] + " de " + jornadaFecha[0][3] )
    else: 
         print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][3] )

def printCurrent(jornadaFecha):
    print("La " + jornadaFecha[0][0] + " se está realizando")
            

if months.index(jornadaFecha[0][3]) > month:
    printFuture(jornadaFecha)
elif months.index(jornadaFecha[0][3]) < month:
    printPast(jornadaFecha)
if months.index(jornadaFecha[0][3]) == month:
    if int(jornadaFecha[0][1]) > day:
        printFuture(jornadaFecha)
    if jornadaFecha[0][2] != '':
        if day <= int(jornadaFecha[0][2]) and  day >= int(jornadaFecha[0][1]):
            printCurrent(jornadaFecha)
        else: 
            printPast(jornadaFecha)
    elif int(jornadaFecha[0][1]) == day:
        printCurrent(jornadaFecha)
    else:
        printPast(jornadaFecha)
        

[('Jornada 26', '01', '04', 'Mar')]
La Jornada 26 se realizó del 01 al 04 de Mar
